In [1]:
print('lala')

lala


In [ ]:
# Подкачка сертификата на экзекьюторы при помощи команды curl
import os
# 0. Определение вспомогательных переменных — имени сертификата и его пароля
ts_name = "..."
ts_password = "..."

# 1. Определение исполняемой команды curl для получения сертификата по указанному адресу
certs = f"curl -vs https://storage.yandexcloud.net/cloud-certs/CA.pem 2>/dev/null | keytool -importcert -alias YandexCA -keystore {ts_name} -storepass {ts_password} -noprompt"

# 2. Выполнение её внутри ОС виртуальных машин
os.system(certs)

# 3. Добавление созданного файла с сертификатом внутрь Spark. Это позволит экзекьюторам обращаться к файлу во время работы
spark.sparkContext.addFile(ts_name)

Certificate was added to keystore


In [ ]:
# Указание имени и пароля для пользователя, которые задавали в предыдущем уроке при создании Kafka-топика.
user = "..."
password = "..."
# Указание адреса Kafka-брокера и имени топика Kafka, которые задавали в предыдущем уроке при создании Kafka-топика.
# Не забудьте добавить номер порта :9091 в конце адреса kafka_broker!
kafka_broker = "..."
kafka_topic = "..."

# Определение шаблона для JAAS-конфига, который будет нужен для аутентификации клиента Spark при подключении к Kafka
jaas_conf_template = "org.apache.kafka.common.security.scram.ScramLoginModule required username='{}' password='{}';"

In [21]:
kafka_df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_broker) \
  .option("kafka.security.protocol", "SASL_SSL") \
  .option("kafka.sasl.mechanism", "SCRAM-SHA-512") \
  .option("kafka.ssl.truststore.location",ts_name) \
  .option("kafka.ssl.truststore.password",ts_password) \
  .option("kafka.sasl.jaas.config", jaas_conf_template.format(user,password)) \
  .option("subscribe", kafka_topic) \
  .option("includeHeaders", "true") \
  .option("startingOffsets", "earliest") \
  .load() \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

kafka_df.show(5, truncate=False)

+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                |
+----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|null|{"HOUSEHOLD_KEY":3,"BASKET_ID":0,"DAY":1,"PRODUCT_ID":2,"QUANTITY":8,"SALES_VALUE":7.25,"STORE_ID":1,"COUPON_MATCH_DISC":13.89,"COUPON_DISC":30.85,"RETAIL_DISC":60.14,"TRANS_TIME":0,"WEEK_NO":1}   |
|null|{"HOUSEHOLD_KEY":3,"BASKET_ID":1,"DAY":1,"PRODUCT_ID":1,"QUANTITY":2,"SALES_VALUE":7.52,"STORE_ID":2,"COUPON_MATCH_DISC":16.67,"COUPON_DISC":34.18,"RETAIL_DISC":63.06,"TRANS_

In [22]:
spark.sql('select count(*) from module_4.transactional_data_1').show()

26/02/12 12:28:37 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+
|count(1)|
+--------+
| 2097686|
+--------+



In [23]:
spark.sql('select count(*) from module_4.transactional_data_1').show()

+--------+
|count(1)|
+--------+
| 2097686|
+--------+

